In [2]:
import csv
from glob import glob
import json
import os
import random

import pandas as pd

In [3]:
random.seed=42

In [1]:
def pass_folder(predictions_folder, label_data):
    for word_path in glob(predictions_folder):
        word = os.path.split(word_path)[-1]
    
        word_uses = pd.read_csv(os.path.join(DWUG_PATH, "data", word, "uses.csv"), sep="\t", quoting=csv.QUOTE_NONE)
    
        clusters_dir = os.path.join(DWUG_PATH, "clusters/opt")
        if not os.path.exists(clusters_dir):  # no opt in RuDSI data
            clusters_dir = os.path.join(DWUG_PATH, "clusters")
        word_clusters = pd.read_csv(os.path.join(clusters_dir, f"{word}.csv"), sep="\t", quoting=csv.QUOTE_NONE)
        word_clusters[CLUSTER_NUMBER_COLUMN] = word_clusters[CLUSTER_NUMBER_COLUMN].astype(int)
    
        
        this_word = word_uses.join(word_clusters.set_index("identifier"), on="identifier")
        #print(this_word.head())
        
        clusters_and_definitions = pd.read_csv(os.path.join(word_path, "cluster_gloss.tsv"), sep="\t")
        clusters_and_definitions["cluster"] = clusters_and_definitions.cluster.astype(int)
        clusters_and_definitions = clusters_and_definitions[clusters_and_definitions.cluster!=-1]
        if clusters_and_definitions.shape[0] > 1:
            
        
            definitions, contexts_list = [], []
            for row in clusters_and_definitions.iterrows():
                cluster_number, definition = row[1]
                
                
                this_cluster = this_word[this_word[CLUSTER_NUMBER_COLUMN] == cluster_number]
                
                contexts = []
        
                for row in this_cluster.iterrows():
                    start, end = row[1]["indexes_target_token"].split(":")
                    start, end = int(start), int(end)
                    example = row[1][EXAMPLE_COLUMN]
                    contexts.append(f"{example[:start]}<b>{example[start:end]}</b>{example[end:]}")
    
                if not contexts: 
                    print(word)
                    print(cluster_number)
                try:
                    contexts_list.append('<br>'.join(set(random.choices(contexts, k=5))))
                except IndexError:
                    print(contexts)
                    raise IndexError
                definitions.append(definition)
            
            for definition in definitions:
                cluster_data = {}
                cluster_data["data"] = {"my_text": f"{word.upper()}: {definition}"}
                cluster_data["data"]["variants"] = [{"html": ctx} for ctx in contexts_list]
                label_data.append(cluster_data)
    return label_data

In [9]:
label_data = []
lang = "en"
DWUG_PATH = os.path.expanduser(f"~/PycharmProjects/gloss-annotator/wugs/dwug_{lang}/")
CLUSTER_NUMBER_COLUMN = 'cluster'
EXAMPLE_COLUMN = 'context'
WORD_COLUMN = 'word'
predictions_folder = os.path.expanduser("~/PycharmProjects/gloss-annotator/predictions/")

label_data = pass_folder(os.path.join(predictions_folder, f"pilot_glmlarge_wordnet_l1norm_top3/dwug_{lang}/*"), label_data)
label_data = pass_folder(
    os.path.join(
    predictions_folder,
    f"pilot_flan-t5-definition-en-xl/dwug_{lang}/*",
    ),
    label_data,
)
label_data = pass_folder(
    os.path.join(
    predictions_folder,
    f"lesk/dwug_{lang}/*",
    ),
    label_data,
)
random.shuffle(label_data)
with open(os.path.expanduser(f"~/PycharmProjects/label-studio-{lang}.json"), "w") as f:
    json.dump(label_data, f)

Labeling interface - https://github.com/ltgoslo/gloss-annotator/blob/main/wugs/label_studio_data/labeling_config.xml